In [2]:
#Downloading the dogs and cats dataset from Kaggle
!kaggle competitions download -c dogs-vs-cats -p ./data/

100%|███████████████████████████████████████▉| 811M/812M [00:53<00:00, 21.7MB/s]
100%|████████████████████████████████████████| 812M/812M [00:53<00:00, 15.9MB/s]


In [6]:
# Unzip the dataset
import zipfile

file = "data/dogs-vs-cats.zip"
data_location = "data/"

zip_files = ['data/test1.zip','data/train.zip']

In [5]:
# Unzipping
zipped = zipfile.ZipFile(file, 'r')
zipped.extractall(data_location)
zipped.close()

In [7]:
for zip in zip_files:
    zipped = zipfile.ZipFile(zip, 'r')
    zipped.extractall(data_location)
    zipped.close()